In [1]:
from kiteconnect import KiteConnect
from kiteconnect import exceptions

In [2]:
import logging
import requests
import ctypes

In [3]:
import pandas as pd

In [4]:
from time import time, sleep

In [5]:
from datetime import datetime, timedelta

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [8]:
api_key = "f5ean4aaxq3wv9n3"
client_id = "ED5107"
api_secret = "s1kbs5y654zhbpqgd2rvslqc6shx9yh8"

In [9]:
logging.basicConfig(level=logging.DEBUG)
kite = KiteConnect(api_key=api_key)
print(kite.login_url())

https://kite.trade/connect/login?api_key=f5ean4aaxq3wv9n3&v=3


In [10]:
# Get the Request Token from the redirect URL

request_token = "OhM6V72Y2nrhEUhEY5nkQ8xJ1DO5yKsO"

# Get Access Token

In [11]:
#data = kite.generate_session(request_token, api_secret=api_secret)
#access_token = data["access_token"]
access_token = "jxoiE5CJEthXY2F5Zr9AWQwIVVX3RlhQ"
kite.set_access_token(access_token)
print(access_token)

jxoiE5CJEthXY2F5Zr9AWQwIVVX3RlhQ


In [12]:
print(kite.profile())

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /user/profile HTTP/1.1" 200 None


{'user_id': 'ED5107', 'user_type': 'individual', 'email': 'ramanathannalini@gmail.com', 'user_name': 'Nalini Ramanathan', 'user_shortname': 'Nalini', 'broker': 'ZERODHA', 'exchanges': ['BFO', 'CDS', 'BSE', 'NFO', 'MF', 'NSE'], 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'], 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'], 'avatar_url': '', 'meta': {'demat_consent': 'consent'}}


# Get Instrument Tokens

In [13]:
def get_instrument_token(tradingsymbol, instrument_type, exchange):
    instruments = pd.DataFrame(kite.instruments())
    instrument_token = instruments[(instruments['tradingsymbol']==tradingsymbol) & (instruments['instrument_type']==instrument_type) & (instruments['exchange']==exchange)]['instrument_token'].to_list()[0]
    return str(instrument_token)

# Get Historical Data

In [14]:
def get_historical_data(instrument_token, from_date, to_date, interval):
    return pd.DataFrame(kite.historical_data(instrument_token=instrument_token,
                                    from_date=from_date, 
                                   to_date=to_date,
                                   interval=interval))

# Get Exponential Moving Average

In [15]:
def get_exponential_moving_avg(timeseries, span):
    """
    adjust=False specifies that we are interested in the recursive calculation mode.
    """
    timeseries[str(span)+'d_EMAvg'] = timeseries.iloc[:,0].ewm(span=span, adjust=False).mean()
    return timeseries

# Stochastic Momentum Indicator

In [16]:
def get_smi(hist_data, k_period = 10, k_smooth_period = 3, d_smooth_period = 3, d_period = 10):
    close = pd.DataFrame({'date':[datetime.strftime(d, "%Y-%m-%d %H:%M:%S") for d in list(hist_data['date'])],'close':list(hist_data['close'])})
    highest_high = pd.DataFrame(data=list(hist_data['high']), columns=['highest_high']).rolling(k_period, min_periods=1).max()
    lowest_low = pd.DataFrame(data=list(hist_data['low']), columns=['lowest_low']).rolling(k_period, min_periods=1).min()
    mean_hl = pd.DataFrame(data=(highest_high['highest_high']+lowest_low['lowest_low'])*0.5, columns=['mean_hl'])
    hh_minus_ll = pd.DataFrame(data=(highest_high['highest_high']-lowest_low['lowest_low']), columns=['hh_minus_ll'])
    close_minus_mean_hl = pd.DataFrame(data=(close['close']-mean_hl['mean_hl']), columns=['close_minus_mean_hl'])
    emavg_cmmhl = pd.DataFrame(data=list(close_minus_mean_hl['close_minus_mean_hl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_cmmhl'])
    emavg_emavg_cmmhl = pd.DataFrame(data=list(emavg_cmmhl['emavg_cmmhl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_emavg_cmmhl'])
    emavg_hhmll = pd.DataFrame(data=list(hh_minus_ll['hh_minus_ll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_hhmll'])
    emavg_emavg_hhmll = pd.DataFrame(data=list(emavg_hhmll['emavg_hhmll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_emavg_hhmll'])
    smi = pd.DataFrame(data=100*(emavg_emavg_cmmhl['emavg_emavg_cmmhl']/(emavg_emavg_hhmll['emavg_emavg_hhmll']/2)), columns=['smi'])
    signal = pd.DataFrame(data=list(smi['smi'].ewm(span=d_period, adjust=False).mean()), columns=['signal'])
    smi_df = pd.concat([close, smi, signal],axis=1)
    smi_df.set_index('date', drop=True, inplace=True)
    return smi_df

# Zerodha Testing LIVE Deployment TradeLog

In [17]:
def get_quote(exchange, tradingsymbol):
    for _ in range(5):
        try:
            quote = kite.quote(exchange+":"+tradingsymbol)[exchange+":"+tradingsymbol]
            break
        except requests.exceptions.ReadTimeout as e:
            logging.exception("Timeout Error - while getting ltp: \n %s", e)
            logging.info("Retrying")
            sleep(1)
            continue
    return quote

In [25]:
def create_candle(current_price, latest_candle, hist_data, all_prices, first_candle_timestamp, candle):
    # Add latest price to latest_candle data
    latest_candle.loc[len(latest_candle)] = current_price
    
    # get OHLC for latest candle
    price_list = list(latest_candle['price'])
    actual_high = max(price_list)
    actual_low = min(price_list)
    actual_close = price_list[-1]
    actual_open = price_list[0]
    
    # Add to candle data
    hist_data.loc[len(hist_data)] = [first_candle_timestamp, actual_open, actual_high, actual_low, actual_close, 0]
    all_prices = pd.concat([all_prices, latest_candle])
    
    # Update variables for next candle
    first_candle_timestamp = first_candle_timestamp+timedelta(minutes=candle)
    latest_candle = pd.DataFrame(columns=['price'])
    
    return latest_candle, hist_data, all_prices, first_candle_timestamp

In [56]:
def place_trade(trade_log, current_time, current_price, current_smi, current_signal, current_transaction_type):
    if len(trade_log)==0 or trade_log.loc[list(trade_log.index)[-1], 'status']=='closed':
        current_status = 'open'
    elif trade_log.loc[list(trade_log.index)[-1], 'status']=='open':
        current_status = 'closed'
    print(current_time, current_transaction_type, current_price, current_status)
    trade_log.loc[len(trade_log)] = [current_time, current_price, current_smi, current_signal, current_transaction_type, current_status]
    return trade_log, current_price, current_transaction_type, current_status

In [65]:
def calculate_pnl(current_transaction_type, transaction_price, price):
    if current_transaction_type == 'buy':
        pnl = price - transaction_price
    elif current_transaction_type == 'sell':
        pnl = transaction_price - price
    return pnl

# Pre-Flight Checks

### WARNING: Don't forget to update the date and time in 'deply_till' variable

In [21]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20SEPFUT'
instrument_type = 'FUT'
exchange = 'NFO'

# TRADE DETAILS
qty = 25
order_type='MARKET'
product='MIS'
variety='regular'

# CHART SPECIFICATIONS
interval = '3minute'
k_period = 10
k_smooth_period = 3
d_smooth_period = 3
d_period = 10
trend_change_buffer = 1
stop_loss = -100


# DEPLOYMENT DETAILS
deploy_till = datetime(datetime.now().year, datetime.now().month, datetime.now().day, hour=15 , minute=16 , second=0)
candle = 3
transaction_buffer = 3

# THE PAYLOAD

In [22]:
instoken = get_instrument_token(tradingsymbol, instrument_type, exchange)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments HTTP/1.1" 200 870105


In [66]:
first_candle_timestamp = datetime(2020, 9, 18, 12, 42)
output_time = datetime(2020, 9, 18, 13, 0)

In [67]:
test_end = first_candle_timestamp
test_start = first_candle_timestamp - timedelta(days=3)
for _ in range(5):
        try:
            hist_data = get_historical_data(instoken, test_start, test_end, interval)
            break
        except requests.exceptions.ReadTimeout as e:
            logging.exception("Timeout Error - while getting ltp: \n %s", e)
            logging.info("Retrying")
            continue
hist_data = hist_data[:-1]
hist_data.tail()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-09-15+12%3A42%3A00&to=2020-09-18+12%3A42%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 7446


,date,open,high,low,close,volume
370,2020-09-18 12:27:00+05:30,22382.80,22405.3,22382.8,22398.85,15350
371,2020-09-18 12:30:00+05:30,22402.00,22410.0,22386.0,22401.55,14175
372,2020-09-18 12:33:00+05:30,22401.55,22404.0,22389.0,22397.25,11250
373,2020-09-18 12:36:00+05:30,22400.00,22405.0,22396.0,22397.45,10025
374,2020-09-18 12:39:00+05:30,22397.95,22398.4,22350.1,22371.95,60025


In [68]:
next_transaction_time = first_candle_timestamp + timedelta(minutes=candle)
previous_close = list(hist_data['close'])[-1]
trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
smi_log = pd.DataFrame(columns=['time','price','smi','signal','signal_gap'])
all_prices = pd.DataFrame(columns=['price'])
latest_candle = pd.DataFrame(columns=['price'])
current_status='closed'
last_signal = 0
trend_gap = 0
check_gud = True
gap_up = False
gap_down = False

while datetime.strftime(datetime.now(), "%H:%M")!=datetime.strftime(deploy_till-timedelta(minutes=1), "%H:%M"):
    sleep(1)
    timestamp = datetime.now()
    print(timestamp)
    quote = get_quote(exchange, tradingsymbol)
    current_price = quote['last_price']
    
    if current_status=="open":
        pnl = calculate_pnl(current_transaction_type, transaction_price, current_price)
        if pnl<stop_loss:
            if current_transaction_type=="buy":
                transaction_type='sell'
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                last_signal = current_signal
            elif current_transaction_type=="sell":
                transaction_type='buy'
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                last_signal = current_signal
    
    # Exit at end of day
    if datetime.strftime(timestamp, "%H:%M")==datetime.strftime(deploy_till-timedelta(minutes=2), "%H:%M"):
        if current_status=='open':
            if current_transaction_type=='buy':
                transaction_type='sell'
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
            if current_transaction_type=='sell':
                transaction_type='buy'
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
        else:
            break
    
    if (next_transaction_time-timestamp).seconds > transaction_buffer:
        latest_candle.loc[len(latest_candle)] = current_price
    elif (next_transaction_time-timestamp).seconds <= transaction_buffer:
        latest_candle, hist_data, all_prices, first_candle_timestamp = create_candle(current_price, latest_candle, hist_data, all_prices, first_candle_timestamp, candle)
        
        smi_df = get_smi(hist_data, k_period, k_smooth_period, d_smooth_period, d_period)
        current_time = list(smi_df.index)[-1]
        current_price = smi_df.loc[current_time, 'close']
        current_smi = smi_df.loc[current_time, 'smi']
        current_signal = smi_df.loc[current_time, 'signal']
        current_gap = current_signal- smi_df.loc[list(smi_df.index)[-2], 'signal']
        smi_log.loc[len(smi_log)] = [current_time, current_price, current_smi, current_signal, current_gap]
        
        if check_gud:
            if previous_close+100<list(hist_data['close'])[-1]:
                gap_up = True
                gup_points = list(hist_data['close'])[-1]-previous_close
                print("GAP UP!")
            elif previous_close-100>list(hist_data['close'])[-1]:
                gap_down = True
                gdown_points = previous_close-list(hist_data['close'])[-1]
                print("GAP DOWN!")
            else:
                print("NO LONGER GAP UP or DOWN")
            check_gud = False
            
        else:
            if gap_up:
                print("GAP UP!")
                if list(hist_data['low'])[-1]<(previous_close+(0.25*gup_points)):
                    gap_up = False
            elif gap_down:
                print("GAP DOWN!")
                if list(hist_data['high'])[-1]>(previous_close-(0.25*gdown_points)):
                    gap_down = False
            else:
                print("NO LONGER GAP UP or DOWN")
        
        if current_gap>0:
            if current_status=="closed":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='buy'
                    if not gap_down:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                    trend_gap = current_gap
                elif trend_gap*current_gap>=0:
                    last_signal = current_signal
            elif current_status=="open":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    trend_gap = current_gap
                    last_signal = current_signal
                    if not gap_down:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                elif trend_gap*current_gap>=0:
                    last_signal = current_signal
        elif current_gap<0:
            if current_status=="closed":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='sell'
                    if not gap_up:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                    trend_gap = current_gap
                elif current_gap*trend_gap>=0:
                    last_signal = current_signal
            elif current_status=="open":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                    trend_gap = current_gap
                    if not gap_up:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                elif trend_gap*current_gap>=0:
                    last_signal = current_signal
        
        next_transaction_time = first_candle_timestamp + timedelta(minutes=candle)
    
    if datetime.strftime(timestamp, "%H:%M")==datetime.strftime(output_time, "%H:%M"):
        trade_log.to_excel('LIVE Test Deployment Logs\Trade_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_GUDA_Stoploss_100.xlsx')
        smi_log.to_excel('LIVE Test Deployment Logs\SMI_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_GUDA_Stoploss_100.xlsx')
        output_time = output_time + timedelta(hours=1)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-09-18 12:42:25.998133


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-09-18 12:42:28.089980


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


...

DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-09-18 14:16:38.927572


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20SEPFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-09-18 14:16:40.732654


ConnectTimeout: HTTPSConnectionPool(host='api.kite.trade', port=443): Max retries exceeded with url: /quote?i=NFO%3ABANKNIFTY20SEPFUT (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x000001F7F1EAB7C8>, 'Connection to api.kite.trade timed out. (connect timeout=7)'))

In [64]:
trade_log.to_excel('LIVE Test Deployment Logs\Trade_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_GUDA_Stoploss_100.xlsx')
smi_log.to_excel('LIVE Test Deployment Logs\SMI_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_GUDA_Stoploss_100D.xlsx')